In [ ]:
import pandas as pd
import numpy as np
import scipy.io 

%load_ext rpy2.ipython

In [ ]:
%%R 

suppressPackageStartupMessages({
    library(cowplot)
    library(LaCroixColoR)
    library(pals)
    library(see)
    library(tidyverse)
    theme_set(theme_cowplot())
})

options(dplyr.width = Inf)

In [ ]:
ensemble_network_dir = "../data/networks/"
example_benchmark_network56_connectome = f"{ensemble_network_dir}/network56.dat"

network_edges_long = np.loadtxt(example_benchmark_network56_connectome)
network_edges_long_df = pd.DataFrame(network_edges_long, columns=["source", "target", "weight"])